In [1]:
import cv2
import numpy as np 
import argparse
import time

In [2]:
def loadYOLO():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco names.txt", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layers_names = net.getLayerNames()
    output_layers = [layers_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    return net, classes, colors, output_layers

def loadImg(img_path):
    # image loading
    img = cv2.imread(img_path)
    img = cv2.resize(img,(500, 500))
    height, width, channels = img.shape
    return img, height, width, channels

In [3]:
def detectObject(img, net, outputLayers):
    blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(outputLayers)
    return blob, outputs

In [4]:
def get_box_dimensions(outputs, height, width):
    boxes = []
    confs = []
    class_ids = []
    for output in outputs:
        for detect in output:
            scores = detect[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x = int(detect[0] * width)
                center_y = int(detect[1] * height)
                w = int(detect[2] * width)
                h = int(detect[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confs.append(float(confidence))
                class_ids.append(class_id)
    return boxes, confs, class_ids

In [5]:
def drawRectangle(boxes, confs, colors, class_ids, classes, img,img_path): 
    indexes = cv2.dnn.NMSBoxes(boxes, confs, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
            #cv2.putText(img, label, (x, y - 5), font, 1, color, 1)
    cv2.imshow("Image", img)
    file_name = os.path.basename(img_path)
    write_path = "Vehicle Images After Detection/Vehicle detection_"+file_name
    cv2.imwrite(write_path,img)

In [6]:
def detectImg(img_path): 
    model, classes, colors, output_layers = loadYOLO()
    image, height, width, channels = loadImg(img_path)
    blob, outputs = detectObject(image, model, output_layers)
    boxes, confs, class_ids = get_box_dimensions(outputs, height, width)
    drawRectangle(boxes, confs, colors, class_ids, classes, image,img_path)
    while True:
        key = cv2.waitKey(1)
        if key == 27:
            break

In [7]:
# detectImg('C:/Users/Ashit/Desktop/SY/SEM 4/LAB/RDBMS/IA2 IMPLEMENTATION/truck.jpg')
detectImg('C:/Users/Hardik Kapadia/OneDrive - somaiya.edu/Somaiya/SY/Sem IV/RDBMS/IA Paper/Git/Vehicle Images before Detection/trucks.jpg') # Make sure to use an Absolute Path

KeyboardInterrupt: 